.. _labels-and-legends:

# Labels and Legends

Of course, most figures require proper labels before they can be of value, so Toyplot provides several labelling mechanisms to help:

## Axes Labels

First, both :ref:`cartesian-axes` and :ref:`table-axes` provide labelling parameters that can be specified when they are created.  In either case the `label` parameter provides a top-level label for the set of axes:

In [1]:
import numpy
import toyplot

canvas = toyplot.Canvas(width=600, height=300)
canvas.axes(grid=(1,2,0), label="Cartesian Axes").plot(numpy.linspace(0, 1)**2)
canvas.table(grid=(1,2,1), label="Table Axes", data = numpy.random.random((4, 3)));

Naturally, some axes - such as Cartesian axes - allow you to specify additional, axis-specific labels:

In [2]:
canvas = toyplot.Canvas(width=300, height=300)
axes = canvas.axes(label="Cartesian Axes", xlabel="Days", ylabel="Users")
axes.plot(numpy.linspace(0, 1)**2);

## Axes Text

Another option for labelling a plot is to insert text labels using the same domain as the data:

In [3]:
def series(x):
    return numpy.cumsum(numpy.random.normal(loc=0.05, size=len(x)))

numpy.random.seed(1234)
x = numpy.arange(100)
y = numpy.column_stack([series(x) for i in range(5)])

In [4]:
label_style = {"text-anchor":"start", "-toyplot-anchor-shift":"5px"}
canvas, axes, mark = toyplot.plot(x, y)
for i in range(y.shape[1]):
    axes.text(x[-1], y[-1,i], "Series %s" % i, style=label_style)

Note that we are using the last coordinate in each series as the text label coordinate - by default, Toyplot renders text centered on its coordinate, so in this case we've chosen a text style that left-aligns the text and offsets it slightly for clarity.

## Canvas Text

When adding text to axes, you specify the text coordinates using the same domain as your data.  Naturally, this limits the added text to the bounds defined by the axes.  For the ultimate in labeling flexibility, you can add text to the canvas directly, using canvas units, outside and/or overlapping axes:

In [5]:
label_style={"font-size":"18px", "font-weight":"bold"}

canvas = toyplot.Canvas(width=600, height=300)
canvas.axes(grid=(1,2,0)).plot(numpy.linspace(1, 0)**2)
canvas.axes(grid=(1,2,1), yshow=False).plot(numpy.linspace(0, 1)**2)
canvas.text(300, 120, "This label overlaps two sets of axes!", style=label_style);

Please keep in mind when placing labels in canvas coordinates that, unlike Cartesian coordinates, canvas coordinates increase from top-to-bottom.

## Canvas Legends

Last-but-not-least, Toyplot provides (currently experimental) support for graphical legends:

In [40]:
observations = numpy.random.power(2, size=(50, 50))

x = numpy.arange(len(observations))

boundaries = numpy.column_stack(
    (numpy.min(observations, axis=1),
     numpy.percentile(observations, 25, axis=1),
     numpy.percentile(observations, 50, axis=1),
     numpy.percentile(observations, 75, axis=1),
     numpy.max(observations, axis=1)))

fill = ["blue", "blue", "red", "red"]
opacity = [0.1, 0.2, 0.2, 0.1]

canvas = toyplot.Canvas(800, 400)
axes = canvas.axes(grid=(1,5,0,1,0,4))
fill = axes.fill(x, boundaries, fill=fill, opacity=opacity)
mean = axes.plot(x, numpy.mean(observations, axis=1), color="blue")

canvas.legend([
    ("Mean", mean),
    ("First Quartile", "rect", {"fill":"blue", "opacity":0.1}),
    ("Second Quartile", "rect", {"fill":"blue", "opacity":0.2}),
    ("Third Quartile", "rect", {"fill":"red", "opacity":0.2}),
    ("Fourth Quartile", "rect", {"fill":"red", "opacity":0.1}),
    ],
    corner=("top-right", 100, 100, 125),
    );


There are several items to note about Toyplot legends:

* Toyplot legends are not associated with axes.
* Legends are placed on the canvas using the same :ref:`canvas-layout` mechanisms as axes.
* Callers explicitly specify the order and contents of the legend.
* The contents of the legend may include references to marks (such as the mean plot), but the caller may also specify explicit legend contents (the four "rect" marks for the four series in the fill) that don't reference any marks at all.